In [1]:
from drn_interactions.interactions.preprocessors import InteractionsPreprocessor
from drn_interactions.interactions.loaders import BaseShockSlowInteractionsLoader
from drn_interactions.interactions.pairwise import PairwiseCorr
from drn_interactions.io import load_neurons, load_derived_generic
import seaborn as sns
import pandas as pd
import numpy as np
from drn_interactions.interactions.graph_clustering import SpectralCluster, ClusterRunner, ClusterEvaluation, ClusterResults
import warnings
from drn_interactions.config import Config, ExperimentInfo
from tqdm import tqdm

dd = Config.derived_data_dir / "ensembles"
dd.mkdir(exist_ok=True)
sns.set_theme(style="ticks", context="paper")

In [3]:
sessions = ExperimentInfo.foot_shock_sessions_10min

preprocessor = InteractionsPreprocessor(z=True, minmax=False)
evaluator = ClusterEvaluation()

loader_fac = lambda session, block, bin_width: BaseShockSlowInteractionsLoader(
    session_name=session, block=block, bin_width=bin_width,
)
corr_fac = lambda: PairwiseCorr(
    remove_self_interactions=True,
    rectify=True,
    shuffle=False,
    remove_duplicate_combs=True,
)
cluster_fac = lambda: SpectralCluster(
    n_clusters=np.arange(2, 8), spectral_kws={"affinity": "precomputed"}, n_init=3
)

runner_fac = lambda session, block, bin_width: ClusterRunner(
    loader=loader_fac(session=session, block=block, bin_width=bin_width),
    preprocessor=preprocessor,
    affinity_calculator=corr_fac(),
    clusterer=cluster_fac(),
    evalulator=evaluator,
)

In [4]:
ensemble_frames = []
ensemble_stats_frames = []

for bin_width in (0.05, 0.1, 0.5, 1):
    for block in ("pre", "base_shock"):
        for session in tqdm(sessions):

            # calculate ensembles
            runner = runner_fac(session=session, block=block, bin_width=bin_width)
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                try:
                    _, ensembles, _, ensemble_stats = runner.run(10)
                except ValueError as e:
                    print(e)
                    print(session)

            # add metadata and append to list of results
            ensembles = ensembles.assign(session_name=session, block=block, bin_width=bin_width)
            ensemble_stats = ensemble_stats.assign(session_name=session, block=block, bin_width=bin_width)
            ensemble_stats_frames.append(ensemble_stats)
            ensemble_frames.append(ensembles)

df_ensembles = pd.concat(ensemble_frames).reset_index(drop=True)
df_ensembles_stats = pd.concat(ensemble_stats_frames).reset_index(drop=True)

df_ensembles_stats["ensemble_id"] = pd.factorize(
    df_ensembles_stats["ensemble"].astype(str)
    + df_ensembles_stats["session_name"]
    + df_ensembles_stats["block"]
    + df_ensembles_stats["bin_width"].astype(str)
)[0]
df_ensembles_stats["ensemble_id"] = np.where(
    df_ensembles_stats["ensemble"] == -1,
    -1,
    df_ensembles_stats["ensemble_id"],
)
df_ensembles = (
    df_ensembles.merge(
    df_ensembles_stats[["session_name", "ensemble", "ensemble_id", "block", "bin_width"]],
    on=["session_name", "ensemble", "block", "bin_width"]
    )
)

df_ensembles.to_csv(dd / "fs - ensembles.csv", index=False)
df_ensembles_stats.to_csv(dd / "fs - stats.csv", index=False)


 62%|██████▎   | 5/8 [01:14<00:39, 13.11s/it]

attempt to get argmax of an empty sequence
hamilton_37


 62%|██████▎   | 5/8 [01:05<00:34, 11.59s/it]

attempt to get argmax of an empty sequence
hamilton_37


 62%|██████▎   | 5/8 [00:55<00:29,  9.74s/it]

attempt to get argmax of an empty sequence
hamilton_37


 62%|██████▎   | 5/8 [00:53<00:27,  9.33s/it]

attempt to get argmax of an empty sequence
hamilton_37


100%|██████████| 8/8 [01:52<00:00, 14.00s/it]


### Ensemble Properties

Are the properties of ensembles different between the two states?

Properties:
- Size
- Average Weight
- Average Conductivity
- Edge Expantion

In [2]:
from drn_interactions.stats import mannwhitneyu_plusplus
import matplotlib.pyplot as plt
from drn_interactions.plots import PAL_GREY_BLACK

In [3]:
df_stats = (
    load_derived_generic("ensembles/fs - stats.csv")
    .query("bin_width == 1")
)

df_ensembles = (
    load_derived_generic("ensembles/fs - ensembles.csv")
    .query("bin_width == 1")
)
neuron_types = load_derived_generic("neuron_types.csv")
df_response =  load_derived_generic("graph/fs - responders.csv")


In [8]:
descriptor = ClusterResults(
    neuron_types=neuron_types,
)
stats = descriptor.ensemble_results(df_ensembles, df_stats)

slow_entropy = descriptor.ensemble_response_entropy(
    ensemble_res=df_ensembles, 
    df_response=df_response, 
    created_col = "slow_response_entropy",
    df_response_response_col="response_fs_slow"
)

fast_entropy = descriptor.ensemble_response_entropy(
    ensemble_res=df_ensembles, 
    df_response=df_response, 
    created_col = "fast_response_entropy",
    df_response_response_col="response_fs_fast"
)

stats = stats.merge(slow_entropy).merge(fast_entropy)
stats.to_csv(dd / "fs - descripted.csv", index=False)